<a href="https://colab.research.google.com/github/vishnuvryeruva/Tensor-Operations/blob/main/Tensor_in_TensorFlow_2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import time

Overloading

In [ ]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))
print(tf.square(2) + tf.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(13, shape=(), dtype=int32)


In [ ]:
k = tf.matmul([[1]], [[2, 3]])
print(k)
print(k.shape)
print(k.dtype)

tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>


Conversion

In [ ]:
arr = np.ones([3, 3])

print("convert numpy arrays to Tensors automatically")
tensor = tf.multiply(arr, 42)
print(tensor)

print("And NumPy operations")
print(np.add(tensor, 1))

print("The .numpy() method explicitly converts")
print(tensor.numpy())

convert numpy arrays to Tensors automatically
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
And NumPy operations
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
The .numpy() method explicitly converts
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


In [ ]:
k = tf.random.uniform([3, 3])

print("GPU"),
print(tf.config.list_physical_devices("GPU"))

print("Tensor on GPU #0:  "),
print(k.device.endswith('GPU:0'))

GPU
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Tensor on GPU #0:  
True


In [ ]:
def time_matmul(k):
  start = time.time()
  for loop in range(10):
    tf.matmul(k, k)

  result = time.time()-start

  print("10 loops: {:0.2f}ms".format(1000*result))

# Brute execution
print("On CPU:")
with tf.device("CPU:0"):
  k = tf.random.uniform([1000, 1000])
  assert k.device.endswith("CPU:0")
  time_matmul(k)

# Brute execution
if tf.config.list_physical_devices("GPU"):
  print("On GPU:")
  with tf.device("GPU:0"):
    k = tf.random.uniform([1000, 1000])
    assert k.device.endswith("GPU:0")
    time_matmul(k)

On CPU:
10 loops: 677.88ms
On GPU:
10 loops: 1689.89ms


Creating DF

In [ ]:
tes = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])
import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
  f.write("""Line 1
Line 2
Line 3
  """)

ds_file = tf.data.TextLineDataset(filename)

In [ ]:
tes = tes.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

In [ ]:
print('Elements of tes:')
for k in tes:
  print(k)

print('\nElements in ds_file:')
for k in ds_file:
  print(k)

Elements of tes:
tf.Tensor([1 9], shape=(2,), dtype=int32)
tf.Tensor([ 4 16], shape=(2,), dtype=int32)
tf.Tensor([36 25], shape=(2,), dtype=int32)

Elements in ds_file:
tf.Tensor([b'Line 1' b'Line 2'], shape=(2,), dtype=string)
tf.Tensor([b'Line 3' b'  '], shape=(2,), dtype=string)


Operations

In [ ]:
m0 = tf.random.normal(shape=[2, 3])
m1 = tf.random.normal(shape=[3, 5])
e = tf.einsum('ij,jk->ik', m0, m1)
# output[i,k] = sum_j m0[i,j] * m1[j, k]
print(e.shape)

(2, 5)


In [ ]:
e = tf.einsum('ij,jk', m0, m1)  # output[i,k] = sum_j m0[i,j] * m1[j, k]
print(e.shape)

(2, 5)


In [ ]:
u = tf.random.normal(shape=[5])
v = tf.random.normal(shape=[5])
e = tf.einsum('i,i->', u, v)  # output = sum_i u[i]*v[i]
print(e.shape)

()


In [ ]:
u = tf.random.normal(shape=[3])
v = tf.random.normal(shape=[5])
e = tf.einsum('i,j->ij', u, v)  # output[i,j] = u[i]*v[j]
print(e.shape)

(3, 5)


In [ ]:
m = tf.ones(2,3)
e = tf.einsum('ij->ji', m0)  # output[j,i] = m0[i,j]
print(e.shape)

(3, 2)


In [ ]:
m = tf.reshape(tf.range(9), [3,3])
diag = tf.einsum('ii->i', m)
print(diag.shape)

(3,)


In [ ]:
trace = tf.einsum('ii', m)  # output[j,i] = trace(m) = sum_i m[i, i]
assert trace == sum(diag)
print(trace.shape)

()


In [ ]:
s = tf.random.normal(shape=[7,5,3])
t = tf.random.normal(shape=[7,3,2])
e = tf.einsum('bij,bjk->bik', s, t)
# output[a,i,k] = sum_j s[a,i,j] * t[a, j, k]
print(e.shape)

(7, 5, 2)


In [ ]:
s = tf.random.normal(shape=[11, 7, 5, 3])
t = tf.random.normal(shape=[11, 7, 3, 2])
e =  tf.einsum('...ij,...jk->...ik', s, t)
print(e.shape)

(11, 7, 5, 2)


**References**

https://www.tensorflow.org/tutorials/customization/basics

https://www.tensorflow.org/api_docs/python/tf/Operation